## Assignment 1 Task 2

<strong>Name:</strong> Elroy Chua Ming Xuan </br>
<strong>UOW ID: </strong> 7431673 </br>
<strong>Data set: </strong> https://www.kaggle.com/datasets/muhammadshahidazeem/customer-churn-dataset

### Step 1: Implement three DT models by the split criteria of Information Gain, Gain Ratio and Gini Index. 
You can use either binary-split or multiple-split.
Decision Tree models based on Information Gain, Gain Ratio, and Gini Index, you need to implement three different splitting criteria. All these criteria basically help in selecting the best feature for splitting the data.

In [91]:
import pandas as pd
import numpy as np
import sys
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
eps = np.finfo(float).eps
from numpy import log2 as log
import random
from pprint import pprint

In [92]:
# Set warning message filter
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [93]:
df_test = pd.read_csv('customer_churn_dataset-testing-master.csv')
df_train = pd.read_csv('customer_churn_dataset-training-master.csv')

# Concatenate the dataframes
df = pd.concat([df_train, df_test], ignore_index=True)
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505207 entries, 0 to 505206
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         505206 non-null  float64
 1   Age                505206 non-null  float64
 2   Gender             505206 non-null  object 
 3   Tenure             505206 non-null  float64
 4   Usage Frequency    505206 non-null  float64
 5   Support Calls      505206 non-null  float64
 6   Payment Delay      505206 non-null  float64
 7   Subscription Type  505206 non-null  object 
 8   Contract Length    505206 non-null  object 
 9   Total Spend        505206 non-null  float64
 10  Last Interaction   505206 non-null  float64
 11  Churn              505206 non-null  float64
dtypes: float64(9), object(3)
memory usage: 46.3+ MB


In [95]:
df.describe()

,CustomerID,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn
count,505206.000000,505206.000000,505206.000000,505206.000000,505206.000000,505206.000000,505206.000000,505206.000000,505206.000000
mean,200779.451782,39.704172,31.350435,15.714825,3.833317,13.496843,620.072766,14.610581,0.555203
std,137241.343095,12.670577,17.237482,8.619323,3.133603,8.451187,245.319256,8.608286,0.496944
min,1.000000,18.000000,1.000000,1.000000,0.000000,0.000000,100.000000,1.000000,0.000000
25%,63827.250000,29.000000,16.000000,8.000000,1.000000,6.000000,446.000000,7.000000,0.000000
50%,193039.500000,40.000000,32.000000,16.000000,3.000000,13.000000,648.900000,14.000000,1.000000
75%,321645.750000,49.000000,46.000000,23.000000,6.000000,20.000000,824.000000,22.000000,1.000000
max,449999.000000,65.000000,60.000000,30.000000,10.000000,30.000000,1000.000000,30.000000,1.000000


### Train Test Split Function

In [96]:
def train_test_split(df, test_size):
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))
    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    return train_df, test_df

In [97]:
train_df, test_df = train_test_split(df, test_size=0.2)

### Creating Functions

In [98]:
# Check if data is pure
def check_purity(data):
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [99]:
def classify_data(data):
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    return classification

In [100]:
def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):
        values = data[:, column_index]
        unique_values = np.unique(values)
        potential_splits[column_index] = unique_values
    return potential_splits

In [101]:
def split_data(data, split_column, split_value):
    split_column_values = data[:, split_column]
    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values > split_value]
    return data_below, data_above

In [102]:
def calculate_mse(data):
    actual_values = data[:,-1]
    if len(actual_values) == 0:
        mse = 0
    else:
        prediction = np.mean(actual_values)
        mse = np.mean((actual_values - prediction)**2)
    return mse

In [103]:
def calculate_entropy(data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [104]:
def calculate_overall_metric(data_below, data_above, metric_function):
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    overall_metric = (p_data_below * metric_function(data_below) + p_data_above * metric_function(data_above))
    return overall_metric

In [105]:
def determine_best_split(data, potential_splits):
    #Add the model selection
    first_iteration = True
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_metric = calculate_overall_metric(data_below, data_above, metric_function=calculate_entropy)
            if first_iteration or current_overall_metric <= best_overall_metric:
                first_iteration = False
                best_overall_metric = current_overall_metric
                best_split_column = column_index
                best_split_value = value
    return best_split_column, best_split_value

In [106]:
#Calculate Entropy of dataset
def findEntropy(df):
    Class = df.keys()[-1] #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [107]:
#Calculate Entroppy by attribute
def findEntropyAttribute(df, attribute):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name

    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)

    entropy2 = 0
    min_entropy = -999
    min_var_name = ""
    
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy

        if(entropy >= min_entropy):
            min_entropy = entropy
            min_var_name = variable
        
    return abs(entropy2), min_var_name

In [108]:
#Helper function to determine gain ratio
def findSplitInfo(df, attribute):

    # Get the unique values of the attribute
    variables = df[attribute].unique()
    if (len(variables) == 1):
        return 1
    split_info = 0

    for variable in variables:
        d = len(df[df[attribute] == variable])/len(df)
        split_info += -d * log(d)

    return abs(split_info)

In [109]:
#Calculate information gain and return the best splitting node(feature)
def infoGain(df, ratio=False):
    df = pd.DataFrame(df, columns=[COLUMN_HEADERS])
    IG=[]
    var_names=[]
    for key in df.keys()[:-1]:
        val, var_name = findEntropyAttribute(df, key)
        ig = findEntropy(df) - val
        if(ratio):
            split_info = findSplitInfo(df, key)
            gain_ration = ig/split_info
            IG.append(gain_ration)
            var_names.append(var_name)
        else:
            IG.append(ig)
            var_names.append(var_name)
    best_ig_index= np.argmax(IG)
    return best_ig_index, var_names[best_ig_index]

In [110]:
def giniImpurity2(valueCounts):
    n = valueCounts.sum()
    p_sum=0
    for key in valueCounts.keys():
        p_sum = p_sum + (valueCounts[key]/n) * (valueCounts[key]/n)
        gini = 1 - p_sum
    return gini

In [111]:
def giniSplitAtt2(df, attName):
    attValues= df[attName].value_counts()
    gini_A = 0
    min_impurity = 999
    min_impurity_variable = ""
    for key in attValues.keys():
        dfKey = df[df.keys()[-1]][df[attName] == key].value_counts()
        numOfKey = attValues[key]
        n = df.shape[0]
        impurity = ((numOfKey/n) * giniImpurity2(dfKey))
        if(impurity <= min_impurity):
            min_impurity = impurity
            min_impurity_variable = key
        gini_A = gini_A + impurity
    return gini_A, min_impurity_variable

In [112]:
def giniIndex2(df, attributeNames):
    df = pd.DataFrame(df, columns=COLUMN_HEADERS)
    giniAttribute={}
    minValue= sys.maxsize
    counter=0
    for key in attributeNames:
        giniAttribute[key] = [*giniSplitAtt2(df, key)]
        if minValue > giniAttribute[key][0]:
            minValue = giniAttribute[key][0]
            selectedAttribute = counter
            selectedVariable = giniAttribute[key][1]
        counter+=1
    return selectedAttribute, selectedVariable


### Decision Tree Algorithm

In [113]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5, model='infoGain'):
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df

    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data, model)
        return classification
    
    # recursive part
    else:
        counter += 1

        #Based on model, determine the best split and value to split on
        if model == 'infoGain':
            split_column, split_value = infoGain(data)
        elif model == 'gainRatio':
            split_column, split_value = infoGain(data, True)
        elif model == 'gini':
            split_column, split_value = giniIndex2(data, COLUMN_HEADERS[:-1])
        
        data_below, data_above = split_data(data, split_column, split_value)
    
    #determine question
    feature_name = COLUMN_HEADERS[split_column]
    question = "{} <= {}".format(feature_name, split_value)

    #instantiate sub-tree
    sub_tree = {question: []}

    #find answers (recursion)
    yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth, model)
    no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth, model)

    #if the answers are the same, then there is no point in asking the question
    if yes_answer == no_answer:
        sub_tree = yes_answer
    else:
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)
    
    return sub_tree


In [127]:
#Show example of decision tree created
tree = decision_tree_algorithm(train_df, max_depth=3, model='infoGain')
pprint(tree)

KeyboardInterrupt: 

In [ ]:
tree = decision_tree_algorithm(train_df, max_depth=3,model='gainRatio')
pprint(tree)

In [ ]:
tree = decision_tree_algorithm(train_df, max_depth=3, model='gini')
pprint(tree)

### Classification Function

In [116]:
def classify_example(example, tree):
    question = list(tree.keys())[0]

    feature_name, comparison_operator, value = question.split(" ")

    if str(example[feature_name]) == value:
        answer = tree[question][0]
    else:
        answer = tree[question][1]
    
    # base case
    if not isinstance(answer, dict):
        return answer
    #recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [117]:
def make_predictions(df, tree):
    if len(df) != 0:
        predictions = df.apply(predict_example, args=(tree,), axis=1)
    else:
        predictions = pd.Series()
    return predictions

In [118]:
def predict_example(example, tree):
    if not isinstance(tree, dict):
        return tree
    
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    if str(example[feature_name]) == value:
        answer = tree[question][0]
    else:
        answer = tree[question][1]
    
    # base case
    if not isinstance(answer, dict):
        return answer
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

In [125]:
def calculate_accuracy(df, tree):
    df["classification"] = df.apply(classify_example, axis=1)
    df["classification_correct"] = df["classification"] == df["target"]

    accuracy = df["classification_correct"].mean()
    return accuracy

### Train decision tree model

In [120]:
train_df, remain_df = train_test_split(df, test_size=0.4) #splits the data into 60% training and 40% testing
test_df, prune_df = train_test_split(remain_df, test_size=0.5) #splits the remaining 40% into 20% testing and 20% validation

In [128]:
ig_tree = decision_tree_algorithm(train_df, max_depth=10, model='infoGain')
gr_tree = decision_tree_algorithm(train_df, max_depth=10,model='gainRatio')
gini_tree = decision_tree_algorithm(train_df, max_depth=10,model='gini')

KeyboardInterrupt: 

In [ ]:
print(f"Info Gain Tree Accuracy : {calculate_accuracy(test_df,ig_tree)}")
print(f"Gain Ratio Tree Accuracy : {calculate_accuracy(test_df,gr_tree)}")
print(f"Gini Index Tree Accuracy : {calculate_accuracy(test_df,gini_tree)}")

### Post Pruning Functions

In [129]:
def filter_df(df, question): 
    feature, comparison_operator, value = question.split() 
    df_yes = df[df[feature].astype(str) == value] 
    df_no = df[df[feature].astype(str) != value] 
    return df_yes, df_no

In [130]:
def determine_leaf(df_train): 
    return df_train.target.value_counts().index[0]

In [131]:
def determine_errors(df_val, tree): 
    predictions = make_predictions(df_val, tree) 
    actual_values = df_val.target 
    return sum(predictions != actual_values)

In [132]:
def pruning_result(tree, df_train, df_val): 
    leaf = determine_leaf(df_train) 
    errors_leaf = determine_errors(df_val, leaf) 
    errors_decision_node = determine_errors(df_val, tree) 
    if errors_leaf <= errors_decision_node: 
        return leaf 
    else: 
        return tree

In [133]:
def post_pruning(tree, df_train, df_val): 
    question = list(tree.keys())[0] 
    yes_answer, no_answer = tree[question]

    #base case
    if not isinstance(yes_answer, dict) and not isinstance(no_answer, dict):
        return pruning_result(tree, df_train, df_val)
    #recursive case
    else: 
        df_train_yes, df_train_no = filter_df(df_train, question) 
        df_val_yes, df_val_no = filter_df(df_val, question) 
        
        if isinstance(yes_answer, dict): 
            yes_answer = post_pruning(yes_answer, df_train_yes, df_val_yes) 
        if isinstance(no_answer, dict): 
            no_answer = post_pruning(no_answer, df_train_no, df_val_no) 
        tree = {question: [yes_answer, no_answer]} 
        
        return pruning_result(tree, df_train, df_val)

### Determine the best depth for each model
#### Info Gain Tree

In [134]:
metrics = {"max_depth": [], "acc_tree": [], "acc_tree_pruned": []}
for n in range(5, 20): 
    tree = decision_tree_algorithm(train_df, max_depth=n, model='infoGain') 
    tree_pruned = post_pruning(tree, train_df, prune_df) 
    metrics["max_depth"].append(n) 
    metrics["acc_tree"].append(calculate_accuracy(test_df, tree)) 
    metrics["acc_tree_pruned"].append(calculate_accuracy(test_df, tree_pruned)) 

df_metrics = pd.DataFrame(metrics)
df_metrics = df_metrics.set_index("max_depth")


KeyboardInterrupt: 

In [ ]:
df_metrics.plot(figsize=(12,5))

In [ ]:
# Index of best accuracy
index = np.argmax(metrics['acc_tree_pruned'])
accuracy = max(metrics['acc_tree_pruned'])

# Depth at which max pruned tree accuracy is acheived
best_depth = metrics['max_depth'][index]

infoGainAccuracy = [accuracy, best_depth]

print(
    f"Best accuracy for pruned Info Gain Decision Tree is acheived at: \nMax depth : {best_depth}\nAccuracy: {accuracy}")


#### Gain Ratio Tree

In [ ]:
metrics = {"max_depth": [], "acc_tree": [], "acc_tree_pruned": []}
for n in range(5, 20): 
    tree = decision_tree_algorithm(train_df, max_depth=n, model='gainRatio') 
    tree_pruned = post_pruning(tree, train_df, prune_df) 
    
    metrics["max_depth"].append(n) 
    metrics["acc_tree"].append(calculate_accuracy(test_df, tree)) 
    metrics["acc_tree_pruned"].append(calculate_accuracy(test_df, tree_pruned)) 
df_metrics = pd.DataFrame(metrics)
df_metrics = df_metrics.set_index("max_depth")

In [ ]:
df_metrics.plot(figsize=(12, 5))

In [ ]:
#Index of best accuracy
index = np.argmax(metrics['acc_tree_pruned'])
accuracy = max(metrics['acc_tree_pruned'])

#Depth at which max pruned tree accuracy is achieved
best_depth = metrics['max_depth'][index]

gainRatioAccuracy =[accuracy, best_depth]
print(
    f"Best accuracy for pruned Gain Ratio Decision Tree is acheived at: \nMax depth : {best_depth}\nAccuracy: {accuracy}")